# This model predicts the chaces of future heart disease. The data set we are using is https://www.kaggle.com/datasets/mamta1999/cardiovascular-risk-data

This model servers as a proof of concept for how Deep Learning can be used in the medical field. With more data and resources this can prove extremely benifical.

In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Data Processing

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from numpy import argmax

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

df= pd.read_csv('https://raw.githubusercontent.com/ReneMazuela/Datasets/main/cario_vascular.csv')

In [ ]:
# Initialize a new LabelEncoder
le = LabelEncoder()

# Apply LabelEncoder to each of the specified columns
df['sex'] = le.fit_transform(df['sex'])
df['is_smoking'] = le.fit_transform(df['is_smoking'])

In [ ]:
# split into input and output columns
X, y = df.values[:, :-1], df.values[:, -1]

# ensure all data are floating point values
X = X.astype('float32')

In [ ]:
# split into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


(2271, 15) (1119, 15) (2271,) (1119,)


# The Model

In [ ]:
# set number of input features
n_features = X_train.shape[1]
n_features

15

In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping


model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)),
    tf.keras.layers.Dropout(0.9),
    tf.keras.layers.Dense(128, activation='relu',kernel_initializer='he_normal', input_shape=(n_features,)),
    tf.keras.layers.Dropout(0.9),
    tf.keras.layers.Dense(128, activation='relu',kernel_initializer='he_normal', input_shape=(n_features,)),
    tf.keras.layers.Dropout(0.9),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

#I believe in my choice to change the thir dense layer to 256 neuron I can increase the capacity?

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy')
    ]
)
early_stop = EarlyStopping(
    monitor ='val_loss',
    mode='min',
    verbose=1,
    patience=10,
    restore_best_weights=True
    )

model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)

In [ ]:
# evaluate the model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)
# make a prediction
row = [20,4,0,1,1,0,0,1,0,322,100,98,29.77,72,60]
yhat = model.predict([row])

Test Accuracy: 0.841
1/1 [==============================] - 0s 60ms/step


Above we can see that with no Fine tunning, the model predictions are significantly accurate.

#Tuning

In [ ]:
# Save the best model
cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_accuracy:.2f}.hdf5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# Reduce learning rate if no improvement in val_loss
cb_reducelr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    mode='min',
    factor=0.1,
    patience=10,
    verbose=1,
    min_lr=0.00001
)

# Stop training if no improvement in val_accuracy
cb_earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    min_delta=0.001,
    patience=10,
    verbose=1
)

# Save training history to CSV file
cb_csvlogger = tf.keras.callbacks.CSVLogger(
    filename='training_log.csv',
    separator=',',
    append=False
)

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(neurons_1=1, neurons_2=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons_1, input_dim=15, kernel_initializer='uniform', activation='linear'))
    model.add(Dense(neurons_2, input_dim=15, kernel_initializer='uniform', activation='linear'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

neurons_1 = [1, 5, 10, 15, 20, 25, 30, 50, 100]
neurons_2 = [1, 5, 10, 15, 20, 25, 30, 50, 100]
param_grid = dict(neurons_1=neurons_1, neurons_2=neurons_2)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X_train, y_train, callbacks=[cb_earlystop], validation_split=0.2)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 11: early stopping
Best: 0.853369 using {'neurons_1': 1, 'neurons_2': 1}


In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(neurons_1, input_dim=15, kernel_initializer='uniform', activation='linear'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X_train, y_train, callbacks=[cb_earlystop], validation_split=0.2)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))